# Advanced example for running an LCA in AeroMAPS 

> *This notebook requires a valid ecoinvent license. You also need to install the corresponding LCA packages with `pip install --upgrade aeromaps[lca]`*

For advanced users willing to customize the LCA model, this notebook shows how to set up and run an AeroMAPS simulation with a custom LCA model defined in a configuration YAML file. 

The custom LCA model used in this example is defined in the file `data/lca_data/custom_model/custom_lca_model.yaml`. It includes:
- The name of the LCA project (this will install or reuse the corresponding [Brightway](https://docs.brightway.dev/en/latest/) project).
- The LCA database to use (ecoinvent 3.10 cutoff in this example).
- The SSP-RCPs scenario to use for projecting the ecoinvent background data with [premise](https://premise.readthedocs.io/en/latest/index.html) (SSP2 with no climate policy in this example - you may change depending on the latest developments of premise).
- The LCA model definition (also referred to as a parametric inventory), which defines the various datasets to be imported from ecoinvent and how they are connected to the AeroMAPS processes/parameters. You may also tag the datasets with a "phase" attribute to split the LCA results by life cycle phase (as done in this example).
- The impact assessment methods to use (either from LCIA methods defined in Brightway/ecoinvent or custom methods with characterization factors provided in CSV files).

For more information on how to define a custom LCA model through a configuration file, please refer to the [LCA-Modeller](https://github.com/AeroMAPS/lca-modeller) library, a side project of AeroMAPS dedicated to the definition of parametric LCA models.


> ***Note**: The first time you run this notebook, it may take a dozen minutes depending on the complexity of the LCA model, as it will install the ecoinvent and premise databases (unless previously installed) and parametrize the LCA model declared in the configuration file (each time the kernel is restarted). Subsequent runs will be much faster.*



## Import librairies and configuration

In [ ]:
# --- Import libraries ---
%matplotlib widget
import sys
sys.path.insert(0, './utils/')
import pandas as pd
import brightway2 as bw
import lca_algebraic as agb
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import math
import collections
import time
from plots import plot_stacked_evolution_subplots
plt.style.use("bmh")
from aeromaps import create_process

## Set models and create process

This example notebook covers the ICAO's LTAG integrated scenarios. The corresponding configuration files are provided in the `./data/config_files` directory.

For using the default LCA model, you must declare it in the configuration file with the following lines:
```yaml
life_cycle_assessment:
    lca_model_data_file: "../lca_data/custom_model/custom_lca_model.yaml"
    split_by: 'phase'
```

There are two inputs to provide here: 
- `lca_model_data_file`: the path to a `.yaml` file containing the LCA model definition. The YAML format is the main difference with the default LCA model provided as a JSON. You may check both files to see how they differ.
- `split_by` (optional): enables you to split the impacts to get the individual contributions of the air transport components (aircraft production, fuel combustion, etc.). This directly relates to the `custom_attributes` fields declared in the YAML file definition.

In [ ]:
# --- Set AeroMAPS models and create process for IS scenario ---
# Note: first call to LCA module takes a dozen minutes depending on CPU, as it will install ecoinvent/premise databases (unless previsouly installed)
# and parametrize the LCA model declared in LCA configuration file (each time the kernel is restarted).
process = create_process(
    configuration_file="data/config_files/config_is2medium_custom_lca.yaml",  # <-- select LTAG IS scenario (from 0 to 3)
)

## Run AeroMAPS simulation and assessment

In [ ]:
# --- Run assessment ---
start_time = time.time()
process.compute()
process.write_json()
print("--- %s seconds ---" % (time.time() - start_time))

## Visualize LCA results

In [ ]:
# --- Results ---
process_data_vector_outputs = process.data["vector_outputs"]
process_data_float_inputs = process.data["float_inputs"]
process_data_climate = process.data["climate_outputs"]
lca_outputs = process.data["lca_outputs"]
lca_outputs

In [ ]:
# --- Plots ---
plt.close()
plot_stacked_evolution_subplots(lca_outputs)

----------

# Exporting your custom LCA model

If you wish to export your custom LCA model for reuse in other AeroMAPS projects or internal sharing, you may have a look at the [export LCA model notebook](./utils/export_lca_model.ipynb). You will learn how to generate a JSON file similar to the one used in the default LCA module. This JSON file can then be used directly for faster LCA calculations without the need to recompile the model each time. 